In [1]:
import time
import json
import requests
import cloudscraper
import pymysql
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

## 上傳資料庫

In [2]:
def uploadDB(df, table_name):
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    sql_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name))
    df.astype(str).to_sql(name=table_name, con=sql_engine, if_exists = 'append', index=False)
    print('Successfully insert into Bigdata table: ' + table_name)

## 爬取DCARD版位資訊 API 網址：https://www.dcard.tw/service/api/v2

### 全部文章	GET	/posts
### 看板資訊	GET	/forums
### 看板內文章列表	GET	/forums/{看板名稱}/posts
### 文章內文	GET	/posts/{文章ID}
### 文章內引用連結	GET	/posts/{文章ID}/links
### 文章內留言	GET	/posts/{文章ID}/comments

In [3]:
def get_all_forums_and_upload():
    url_path = 'https://www.dcard.tw/service/api/v2/forums'
    df = get_df_from_dcard_api(url_path)
    df = df[['id', 'alias', 'name', 'subscriptionCount', 'postCount', 'isSchool', 'createdAt']]
    df.postCount = df.postCount.apply(lambda x : x['last30Days'])
    df.createdAt = df.createdAt.apply(lambda x : x[:10])
    # uploadDB(df, 'dcard_forums')
    return df

In [4]:
def get_df_from_dcard_api(url):
    response = requests.get(url).text
    # scraper = cloudscraper.create_scraper()
    # response = scraper.get(url).text
    ##
    data = json.loads(response)
    df = pd.DataFrame(data)
    return df

In [5]:
def get_latest_post_from_forums(forums_name, post_count, popular):
    postid = ''
    max_limit = 100 #一次拿100篇(上限)
    iteration = int(post_count/max_limit)
    df_post = pd.DataFrame()
    for i in range(0,iteration):
        post_url = 'https://www.dcard.tw/service/api/v2/forums/' + forums_name + '/posts?popular=' + popular + '&limit=' + str(max_limit)
        suffix = '&before=' + str(postid)
        if postid != '':
            post_url = post_url + suffix
        df_post_batch = get_df_from_dcard_api(post_url)
        df_post = df_post.append(df_post_batch)
        postid = df_post_batch.tail(1).id.tolist()[0]
        # time.sleep(3)
    return df_post

In [6]:
# 建立版位資訊表
df_forums = get_all_forums_and_upload()

In [7]:
df_forums = df_forums[(df_forums.isSchool == False) & (df_forums.postCount >= 1000)]
df_forums = df_forums.sort_values(by = 'postCount', ascending=False)
df_forums.head()

,id,alias,name,subscriptionCount,postCount,isSchool,createdAt
370,e33fd5d2-025f-46e4-8b81-3f6fecdd25e1,whysoserious,廢文,35741,41381,False,2020-02-04
228,42851318-b9e2-4a75-8a05-9fe180becefe,relationship,感情,576409,10532,False,2020-02-04
229,75a726e6-d4e3-4902-a410-2430a39fffcb,mood,心情,277869,8909,False,2020-02-04
230,255fd275-fec2-49d2-8e46-2e1557ffaeb0,talk,閒聊,362319,6078,False,2020-02-04
373,1ce3ebca-8701-42d5-b14c-076fc629bc8e,sex,西斯,633070,5289,False,2020-02-04


In [92]:
forums_list = df_forums.alias.tolist()
print(forums_list)

['whysoserious', 'relationship', 'mood', 'talk', 'sex', 'job', 'makeup', 'freshman', 'trending', 'sportsevents', 'exam', 'girl', 'aov', 'animal_crossing', 'rainbow', 'meme', 'horoscopes', 'food', '3c', '2019_ncov', 'dressup', 'pet', 'buyonline', 'vehicle', 'illustration', 'savemoney', 'funny', 'apple', 'entertainer', 'stock', 'acg', 'game']


In [103]:
# 頻繁的呼叫API會被擋
# 擷取最熱門版位，最近100篇發文資訊，並上傳
# forums_list = ['game']
for forums_name in forums_list:
    print(forums_name)
    df_post = get_latest_post_from_forums(forums_name = forums_name, post_count = 100, popular = 'true')
    df_post = df_post[['id', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'school', 'gender', 'createdAt']]
    df_post.columns = ['postid', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'author', 'gender', 'createdAt']
    uploadDB(df_post, 'dcard_posts')
    time.sleep(3)

game
Successfully insert into Bigdata table: dcard_posts


### 爬留言

In [12]:
pd.set_option("display.max_columns", None)

In [20]:
postid = 236804037
url = 'https://www.dcard.tw/service/api/v2/posts/' + str(postid) + '/comments'

In [28]:
df_comments = get_df_from_dcard_api(url)
df_comments_tmp = df_comments[['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department']]

In [29]:
df_comments_tmp

,id,anonymous,postId,createdAt,floor,content,likeCount,withNickname,gender,school,department
0,ce81ebdd-80d2-4948-b79d-88f211fe5d88,False,236804037,2021-08-25T06:06:50.402Z,1,(((o(*ﾟ▽ﾟ*)o))) 我是不是該去跟笨蛋們收帳呢？,2,True,M,善良仔米蟲,dionysos
1,c7ca0454-9960-4e6e-920f-1366030ac819,True,236804037,2021-08-25T06:07:45.033Z,2,不知道為什麼有點想哭🥲,2,False,F,淡江大學,NaN
2,d0a7dc32-238a-402d-8d1c-a61ff68ef06a,False,236804037,2021-08-25T06:13:44.743Z,3,:0,1,True,M,柴奴寶ˁ ˙꒳​˙‎ˀ,little_shiba_nu
3,b3cf3ae1-b07e-4a0b-bc75-0a9b167f49fc,False,236804037,2021-08-25T06:14:01.572Z,4,9/6有可能降1級嗎,0,True,M,柴奴寶ˁ ˙꒳​˙‎ˀ,little_shiba_nu
4,26a8e59f-25f2-496f-b01a-1a94f725fdcd,True,236804037,2021-08-25T06:15:03.639Z,5,終於欸 666,1,False,M,長榮大學,NaN
5,41ddf582-1c63-4815-8f45-04c55b9b31a8,False,236804037,2021-08-26T05:10:12.799Z,6,大家都好棒(๑و•̀ㅂ•́๑)و✧,0,False,M,國立彰化師範大學,物理學系
